In [1]:
import pandas
!pip install datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("MedRAG/textbooks")

# ✅ Convert to Pandas DataFrame (Modify split if needed)
df = pd.DataFrame(dataset['train']).head(30000)

if 'contents' in df.columns:
    df = df.drop(columns=['contents'])

print("📊 Dataset Preview:\n", df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Cell_Biology_Alberts.jsonl:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Anatomy_Gray.jsonl:   0%|          | 0.00/5.19M [00:00<?, ?B/s]

...

Pharmacology_Katzung.jsonl:   0%|          | 0.00/11.9M [00:00<?, ?B/s]

Surgery_Schwartz.jsonl:   0%|          | 0.00/30.1M [00:00<?, ?B/s]

Psichiatry_DSM-5.jsonl:   0%|          | 0.00/6.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/125847 [00:00<?, ? examples/s]

📊 Dataset Preview:
                id         title  \
0  Anatomy_Gray_0  Anatomy_Gray   
1  Anatomy_Gray_1  Anatomy_Gray   
2  Anatomy_Gray_2  Anatomy_Gray   
3  Anatomy_Gray_3  Anatomy_Gray   
4  Anatomy_Gray_4  Anatomy_Gray   

                                             content  
0  What is anatomy? Anatomy includes those struct...  
1  Observation and visualization are the primary ...  
2  How can gross anatomy be studied? The term ana...  
3  This includes the vasculature, the nerves, the...  
4  Each of these approaches has benefits and defi...  


In [ ]:
def preprocess_data(df):
    df_cleaned = df.rename(columns={
        "title": "title",
        "content": "abstract"  # Map `content` to `abstract`
    })
    df_cleaned["source"] = df_cleaned["title"]  # Use `title` as `source`
    df_cleaned["id"] = df_cleaned.index.astype(str)  # Generate IDs
    return df_cleaned[["id", "title", "abstract", "source"]]

df_cleaned = preprocess_data(df)

In [3]:
def encode_and_store(df, batch_size=500):
    """Encodes dataset text using BioBERT and stores in Qdrant in smaller batches."""
    num_batches = int(np.ceil(len(df) / batch_size))  # Calculate number of batches

    for i in range(num_batches):
        batch_df = df[i * batch_size: (i + 1) * batch_size]  # Get batch
        points = []

        for _, row in batch_df.iterrows():
            text = f"{row['title']} {row['abstract']}"
            vector = bioBERT_model.encode(text).tolist()
            unique_id = str(uuid.uuid4())  # Generate a unique UUID

            points.append(PointStruct(id=unique_id, vector=vector, payload=row.to_dict()))

        client.upsert(collection_name=COLLECTION_NAME, points=points)  # Store batch
        print(f"✅ Stored batch {i+1}/{num_batches} ({len(batch_df)} records)")


In [ ]:
encode_and_store(df_cleaned, batch_size=500)